In [ ]:
# initial install
%pip install pandas
%pip install openpyxl

In [ ]:
# install packages
%pip install -r requirements.txt

In [ ]:
# update packages
%pip freeze > requirements.txt

In [ ]:
# import packages + load dataframe
import pandas as pd

# Load CSV file into DataFrame
df = pd.read_csv('data.csv')


# Data Exploration

In [ ]:
# analyze dataframe

print(list(df.columns))

In [ ]:
# View ranges of data in each column

for col in df.columns:
    unique_values = df[col].dropna().unique()  # Drop NaNs if you don’t want them in the list
    unique_values_list = [float(val) if isinstance(val, (float, int)) else val for val in unique_values]
    print(f"{col}: {unique_values_list}")


# Clean Dataframe

In [ ]:
# 1. Drop rows that aren't questions, or questions that have text values.
columns_to_keep = [col for col in df.columns if ('Q' in col or 'QN' in col) and 'TEXT' not in col]
columns_to_drop = [col for col in df.columns if col not in columns_to_keep]
df = df[columns_to_keep]

# Print dropped columns
print("Dropped columns:", columns_to_drop)

In [ ]:
# 2. Drop all QN columns

columns_to_drop = [col for col in df.columns if col.startswith("QN")]
columns_to_keep = [col for col in df.columns if col.startswith("Q") and not col.startswith("QN")]
df_old = df[columns_to_drop]
df = df[columns_to_keep]

print("Dropped columns:", columns_to_drop)
print("\n")
print("Kept columns:", columns_to_keep)


In [9]:
# 3. Merge all multi-choice questions into one column.

In [ ]:
# 3: Question 4a

# Debug print to display the value counts of Q4a before transformations
print("Initial value counts in Q4a:")
print(df['Q4a'].value_counts(dropna=False))
print("Initial value counts in Q4b:")
print(df['Q4b'].value_counts(dropna=False))

# Transformation code
main_column = 'Q4a'
additional_columns = ['Q4b', 'Q4c', 'Q4d', 'Q4e']

for index, row in df.iterrows():
    for col in additional_columns:
        try:
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1]  # Use suffix (b, c, d, or e)
                break
        except (ValueError, TypeError):
            continue

df = df.drop(columns=additional_columns)

# Display the updated value counts in Q4a after transformations
print("Updated value counts in Q4a:")
print(df['Q4a'].value_counts(dropna=False))

In [ ]:
# 3: Question 5

# Debug print to display the value counts of QN5A before transformations
print("Initial value counts in QN5A:")
print(df['Q5a'].value_counts(dropna=False))

# Define the main column and additional columns to check for QN5
main_column = 'Q5'
additional_columns = ['Q5a', 'Q5b', 'Q5c', 'Q5d', 'Q5e']

# Initialize QN5 column with values from QN5A where appropriate
df[main_column] = df['Q5a']

# Transformation logic for QN5
for index, row in df.iterrows():
    for col in additional_columns:
        if not pd.isnull(row[col]) and row[col] != 0:
            df.at[index, main_column] = col[-1]  # Set QN5 to the last character of the column name (e.g., 'A' for QN5A)
            break

# Drop the additional columns
df = df.drop(columns=additional_columns)

# Display the updated value counts in QN5 after transformations
print("Updated value counts in Q5:")
print(df['Q5'].value_counts(dropna=False))


In [ ]:
# 3: Question 11

# Debug print to display the initial value counts in Q11A before transformation
print("Initial value counts in Q11A:")
print(df['Q11a'].value_counts(dropna=False))

# Define the main column and all Q11* columns to check for Q11
main_column = 'Q11'
additional_columns = ['Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'Q11f', 'Q11g', 'Q11h', 'Q11i', 'Q11j', 'Q11k', 'Q11l', 'Q11m', 'Q11n']

# Initialize Q11 column with values from Q11A where appropriate
df[main_column] = df['Q11a']

# Transformation logic for Q11
for index, row in df.iterrows():
    for col in additional_columns:
        if not pd.isnull(row[col]) and row[col] != 0:
            df.at[index, main_column] = col[-1].lower()  # Set Q11 to the last character of the column name in lowercase
            break

# Drop all Q11* columns, leaving only Q11
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q11 after transformations
print("Updated value counts in Q11:")
print(df['Q11'].value_counts(dropna=False))

In [ ]:
# Question 12

# Debug print to display the initial value counts for each relevant column before transformation
print("Initial value counts in Q12a:")
print(df['Q12a'].value_counts(dropna=False))

# Define the main column and columns to check
main_column = 'Q12'
additional_columns = ['Q12a', 'Q12b', 'Q12c', 'Q12d', 'Q12e', 'Q12f', 'Q12g', 'Q12h', 'Q12i', 'Q12j', 'Q12k', 'Q12l', 'Q12m', 'Q12n']

# Initialize Q12 with NaN or a default value
df[main_column] = None

# Transformation logic for Q12
for index, row in df.iterrows():
    assigned_value = False
    # Check each column in additional_columns for 1.0, Z, or N values
    for col in additional_columns:
        if row[col] == 1.0:
            df.at[index, main_column] = "NC1"  # Set Q12 to "NC1" if any column has 1.0
            assigned_value = True
            break
        elif not pd.isnull(row[col]) and row[col] in ["Z", "N"]:
            df.at[index, main_column] = row[col]  # Assign "Z" or "N" if present in any column
            assigned_value = True
            break

# Drop all Q12* columns, leaving only Q12
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q12 after transformations
print("Updated value counts in Q12:")
print(df['Q12'].value_counts(dropna=False))